<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/40_TorchScript_Scripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 40: TorchScript Scripting (Kompilacja Kodu)

W poprzedniej lekcji `trace` zepsuł nasz model z `if`-em.
Rozwiązaniem jest **`torch.jit.script`**.

**Scripting** nie uruchamia modelu na próbę. On **analizuje kod źródłowy** (AST - Abstract Syntax Tree).
Widzi `if x > 0:` i tłumaczy to na język TorchScript (IR), zachowując logikę warunkową.

**Wymagania:**
Aby kod dał się skompilować, musi być napisany w "podzbiorze Pythona" (TorchScript Language).
*   Wszystkie typy muszą być jawne (Type Hinting).
*   Nie można używać niektórych dynamicznych funkcji Pythona (np. `try-except`, dynamiczne listy różnych typów).

In [1]:
import torch
import torch.nn as nn

# Ten sam model co wcześniej
class DynamicNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10, 10)

    # WAŻNE: W Scriptingu warto używać Type Hinting!
    # Mówimy wprost: x to Tensor, zwracamy Tensor.
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.sum() > 0:
            return self.linear(x) * 2
        else:
            return self.linear(x) - 100

model = DynamicNet()
print("Model gotowy.")

Model gotowy.


## Kompilacja (`torch.jit.script`)

Tym razem nie podajemy danych przykładowych!
Podajemy sam model.

In [2]:
# SCRIPTING (Kompilacja kodu)
scripted_model = torch.jit.script(model)

print("✅ Model skompilowany (Scripting).")
print(type(scripted_model)) # RecursiveScriptModule

✅ Model skompilowany (Scripting).
<class 'torch.jit._script.RecursiveScriptModule'>


## Inspekcja Kodu (IR)

Zobaczmy `scripted_model.code`.
Tym razem powinieneś zobaczyć instrukcję `if` wewnątrz skompilowanego kodu!
(Będzie wyglądać trochę dziwnie, np. `prim::If`, ale tam będzie).

In [3]:
print("--- KOD TORCHSCRIPT ---")
print(scripted_model.code)

print("\nWidzisz instrukcję 'if'? To znaczy, że logika została zachowana!")

--- KOD TORCHSCRIPT ---
def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    linear = self.linear
    _0 = torch.mul((linear).forward(x, ), 2)
  else:
    linear0 = self.linear
    _1 = torch.sub((linear0).forward(x, ), 100)
    _0 = _1
  return _0


Widzisz instrukcję 'if'? To znaczy, że logika została zachowana!


## Dowód Poprawności

Sprawdźmy, czy model działa poprawnie zarówno dla danych dodatnich, jak i ujemnych.

In [4]:
# Dane dodatnie (ścieżka A)
pos = torch.ones(1, 10)
out_pos_py = model(pos)
out_pos_jit = scripted_model(pos)

# Dane ujemne (ścieżka B - ta, która w Tracingu nie działała)
neg = -torch.ones(1, 10)
out_neg_py = model(neg)
out_neg_jit = scripted_model(neg)

print("Test Positive:")
if torch.allclose(out_pos_py, out_pos_jit):
    print("✅ Zgadza się.")

print("\nTest Negative (To wcześniej nie działało):")
if torch.allclose(out_neg_py, out_neg_jit):
    print("✅ Zgadza się! If działa.")
else:
    print("❌ Błąd.")

Test Positive:
✅ Zgadza się.

Test Negative (To wcześniej nie działało):
✅ Zgadza się! If działa.


## Type Hinting: Pułapka

Scripting jest restrykcyjny.
Jeśli masz funkcję, która czasem zwraca `Tensor`, a czasem `List[Tensor]`, kompilator rzuci błędem.
Musisz używać `Union`, `List`, `Tuple`, `Dict` z modułu `typing`.

In [5]:
from typing import List, Dict

class StrictNet(nn.Module):
    def __init__(self):
        super().__init__()
    
    # Musimy powiedzieć kompilatorowi, co wchodzi i co wychodzi
    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        result = x * 2
        # Zwracamy słownik
        return {"output": result}

try:
    s_net = torch.jit.script(StrictNet())
    print("✅ Udało się skompilować model z typami.")
    
    # Test
    out = s_net(torch.ones(5))
    print(out["output"])
    
except Exception as e:
    print(f"Błąd kompilacji: {e}")

✅ Udało się skompilować model z typami.
tensor([2., 2., 2., 2., 2.])


## 🥋 Black Belt Summary

1.  **`trace` vs `script`:**
    *   Używaj `trace` zawsze, gdy możesz (jest prostszy, obsługuje więcej bibliotek Pythonowych).
    *   Używaj `script` tylko wtedy, gdy masz `control flow` (if, for) zależne od danych wejściowych.
2.  **Mieszanie:** Możesz mieszać obie metody!
    ```python
    @torch.jit.script
    def complex_logic(x):
        if x > 0: return x
        else: return -x

    class MyModel(nn.Module):
        def forward(self, x):
            x = complex_logic(x) # To jest skryptowane
            return self.layer(x) # Resztę można trace'ować
    ```
3.  **Deployment:** Skompilowany model (`.save()`) można załadować w C++ używając `torch::jit::load()`. Tak działa AI w samochodach autonomicznych i robotach.